In [32]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt


## Cargar el audio 

In [33]:
audio_path = 'Menu.mp3'

## Representación digital

En este paso se convierte el archivo en una representación digital que se puede analizar. La fragmentación ocurre después, cuando realizas análisis como el cálculo de RMSE, MFCC, etc.

In [34]:
signal, sr = librosa.load(audio_path, sr=None)
# sr = none siginifica que no se va a cambiar la frecuencia de muestreo
# si sr = 22050, se va a cambiar la frecuencia de muestreo a 22050 Hz
print(f"Signal shape: {signal.shape}, Sample Rate: {sr}")

Signal shape: (1411200,), Sample Rate: 44100


## Root Mean Square Energy (RMSE)
Mide la intensidad (o energía) de la señal en cada fragmento.

¿Por qué calcular RMSE?
    - Te ayuda a analizar dónde está la energía de la canción. Por ejemplo, los momentos con mayor RMSE suelen corresponder a golpes de batería o transiciones importantes.
    - Es útil para entender la estructura dinámica de la canción (partes suaves vs. intensas).

In [35]:
rmse = librosa.feature.rms(y=signal)
print(f'RMSE: {rmse}')

RMSE: [[0.12341513 0.15349023 0.17858239 ... 0.         0.         0.        ]]


## Zero-Crossing Rate

indica el número de veces que una señal cruza el eje horizontal, es decir, el número de veces que la amplitud alcanza 0.

In [36]:
#Esto devolverá el número total de veces que la amplitud cruza el eje horizontal.
Zero_crossing = sum(librosa.zero_crossings(signal, pad=False))
print(f'Zero crossing: {Zero_crossing}')


Zero crossing: 236501


## Tempo 

El tempo es la velocidad o ritmo de una pieza musical y se mide en BPM (beats per minute). Es una característica importante para clasificar canciones por género, por ejemplo, el tempo de una canción de rock suele ser más rápido que el de una balada.

In [37]:
tempo = librosa.beat.tempo(y=signal, sr=sr)
print(f'Tempo: {tempo}')

/tmp/ipykernel_775/2445573866.py:1: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(y=signal, sr=sr)


Tempo: [120.18531977]


# MFCC
- Fragmenta el audio 
    - Divide la señal en fragmentos (ventanas de tiempo), generalmente de 20-40 ms, porque en este intervalo el audio se considera estacionario.
    - Esto se basa en la frecuencia de muestreo del audio. Por ejemplo, si la frecuencia de muestreo es de 22050 Hz, un fragmento de 20 ms contendrá 441 muestras.
- Calculo de las caracteristicas 
    - por cada fragmento , se genera un vector de características que representa las propiedades de la señal en ese fragmento.
    - Cada vector de características se calcula utilizando la transformada de Fourier de tiempo corto (STFT) y luego se aplica la transformada de coseno discreta (DCT) para obtener los coeficientes MFCC.

In [38]:
mfccs = librosa.feature.mfcc(y=signal, sr=sr , n_mfcc=13)
# n_mfcc = 13, significa que se van a extraer 13 coeficientes MFCC
# por defecto n_mfcc = 20
print(f'MFCCs: {mfccs.shape}')

MFCCs: (13, 2757)


### significado de los resultados (shape)
- (n_mfcc, t) donde n_mfcc es el número de coeficientes MFCC y t es el número de fragmentos.
    - 13 coeficientes MFCC es un valor común, pero puedes elegir cualquier valor según tus necesidades.
    - los coeficientes MFCC forman las dimensiones del vector característico para un fragmento del audio.
    - t depende de la duración del audio y del tamaño de la ventana. Por ejemplo, si el audio dura 30 segundos y la ventana es de 20 ms, t será 1500 (30*1000/20).


In [ ]:
# array de 13 x 1292 = 16796
    # en total hay 16796 coeficientes MFCCs o características extraídas
    # el total de dimenciones de las características extraídas es 16796
    
for i in mfccs:
    print(i)

[-184.21811 -188.84097 -298.82767 ... -602.1457  -602.1457  -602.1457 ]
[105.71151   80.541756 -10.618538 ...   0.         0.         0.      ]
[16.828331 25.798645 20.893356 ...  0.        0.        0.      ]
[34.55933  32.73453  30.238724 ...  0.        0.        0.      ]
[ 4.0832405  3.6186526 -6.8369436 ...  0.         0.         0.       ]
[28.424076 29.496525 19.031963 ...  0.        0.        0.      ]
[ 4.545714   2.8280363 -5.0747976 ...  0.         0.         0.       ]
[20.061417 20.895676 15.281046 ...  0.        0.        0.      ]
[ -8.686144 -11.17137  -19.600624 ...   0.         0.         0.      ]
[ -8.1194725  -9.662573  -24.355122  ...   0.          0.
   0.       ]
[-39.448494 -43.68834  -60.609154 ...   0.         0.         0.      ]
[-32.705643 -36.80596  -53.114727 ...   0.         0.         0.      ]
[-36.147396 -40.43567  -50.355484 ...   0.         0.         0.      ]


### Spectal Features 
- **Spectral Centroid**: indica en qué frecuencia se centra la masa de energía de la señal.
- Spectral Rolloff: es la frecuencia por debajo de la cual se encuentra el 85% de la energía de la señal.
- **Spectral Bandwidth**: es el ancho de banda de la señal.
- **Spectral Contrast**: mide la diferencia de amplitud entre picos y valles en el espectro de frecuencia.
- **Spectral Flatness**: mide la uniformidad de la distribución espectral.
- **Spectral Rolloff**: es la frecuencia por debajo de la cual se encuentra el 85% de la energía de la señal.
- Chroma Frequencies: mide la presencia de diferentes notas musicales en la señal.
- Tonnetz: mide la relación entre las notas musicales en la señal.


In [41]:
spectral_centroids = librosa.feature.spectral_centroid(y=signal, sr=sr)[0]
spectral_bandwidth=librosa.feature.spectral_bandwidth(y=signal, sr=sr)[0]
spectral_contrast=librosa.feature.spectral_contrast(y=signal, sr=sr)[0]
spectral_flatness=librosa.feature.spectral_flatness(y=signal)[0]
spectral_rolloff=librosa.feature.spectral_rolloff(y=signal, sr=sr)[0]
print (f'Spectral Centroids: {spectral_centroids}')
print (f'Spectral Bandwidth: {spectral_bandwidth}')
print (f'Spectral Contrast: {spectral_contrast}')
print (f'Spectral Flatness: {spectral_flatness}')
print (f'Spectral Rolloff: {spectral_rolloff}')


Spectral Centroids: [2993.40307513 3146.53471881 3416.80047901 ...    0.            0.
    0.        ]
Spectral Bandwidth: [4200.05518335 4320.00072743 4521.97892848 ...    0.            0.
    0.        ]
Spectral Contrast: [ 0.80480367  0.9205368  12.15060782 ... 11.96340468 11.96340468
 11.96340468]
Spectral Flatness: [9.3464809e-04 2.1910659e-04 7.5828007e-06 ... 1.0000007e+00 1.0000007e+00
 1.0000007e+00]
Spectral Rolloff: [6675.29296875 7041.35742188 7472.02148438 ...    0.            0.
    0.        ]
